In [1]:
import numpy as np
import pImpactR as impact
from copy import deepcopy as copy
import matplotlib.pyplot as plt
from math import ceil
import os

In [2]:
NTURN = [128,256,512,1024]
nTurn = NTURN[3]
pipe_radius = 0.02*np.sqrt(2.0)
nptTest = 3834
npt_yup = 512174

# 1. Preparation

In [3]:
beam, lattice = impact.readInputFile('test_IOTA.in')
ke   = beam.kinetic_energy
mass = beam.mass
freq = beam.frequency

reading ImpactZ input file (test_IOTA.in)
  : mpi task info .............................done
  : simulation control parameters .............done
  : space charge field solver, mesh info ......done
  : dist-type,restart,subcycle,#of state ......done
  : Multiple Charge State info ................done
  : particle distribution info ................done
  : beam reference orbit info .................done
  : converting impact dist to twiss param......done
  : lattice info ..............................done


### adjust lattice

In [4]:
for i in range(len(lattice)):
    if 'pipe_radius' in lattice[i].keys() and lattice[i].type!='dipole':
        lattice[i].pipe_radius = pipe_radius
        
lattice = [ item for item in lattice if not item.type == 'write_raw_ptcl' ]
lattice = [ item for item in lattice if not item.type == '-8' ]
lattice = [ item for item in lattice if not item.type == 'multipole_thin' ]


TBTsplit = impact.getElem('TBT_multiple_file')
TBTsplit.file_id = nTurn+1+100000
TBTsplit.pID_begin = nptTest+1
TBTsplit.pID_end   = nptTest+npt_yup
TBTsplit.n_files   = 32
lattice.insert(1,TBTsplit)

TBT = impact.getElem('TBT')
TBT.pID_begin = 1
TBT.pID_end   = nptTest
TBT.file_id = nTurn+100000
lattice.insert(1,TBT)


L=0.0
for i in range(len(lattice)):
    if 'length' in lattice[i].keys():
        L = L+lattice[i].length
print(L)

39.968229715800106


### backward lattice

In [5]:
latticeB = copy(lattice[::-1])
latticeB.insert(0,copy(lattice[0]))
latticeB.pop()
latticeB[0].turns = nTurn

for i in range(len(latticeB)):
    if 'length' in latticeB[i]:
        latticeB[i].length = -latticeB[i].length
    if latticeB[i].type == 'dipole':
        latticeB[i].bending_angle            =-latticeB[i].bending_angle
        latticeB[i].fringe_field_integration =-latticeB[i].fringe_field_integration

In [6]:
i = 34
print(latticeB[-i])
print(lattice[i])


                   length: -0.7330382858 [m]
                 n_sckick: 10 [1]
                    n_map: 20 [1]
            bending_angle: -1.047197551 [rad]
                       k1: 0.0
                  file_id: 350
              pipe_radius: 0.058 [m]
           entrance_angle: 0.0 [rad]
               exit_angle: 0.0 [rad]
       entrance_curvature: 0.0 [rad]
           exit_curvature: 0.0 [rad]
 fringe_field_integration: -0.5 [1]
                     type: 'dipole'

                   length: 0.7330382858 [m]
                 n_sckick: 10 [1]
                    n_map: 20 [1]
            bending_angle: 1.047197551 [rad]
                       k1: 0.0
                  file_id: 350
              pipe_radius: 0.058 [m]
           entrance_angle: 0.0 [rad]
               exit_angle: 0.0 [rad]
       entrance_curvature: 0.0 [rad]
           exit_curvature: 0.0 [rad]
 fringe_field_integration: 0.5 [1]
                     type: 'dipole'


### write Impact input file

In [7]:
beam.distribution.distribution_type = 'ReadFile_binary'
beam.distribution.file_id = nTurn
beam.nCore_y = 48
beam.nCore_z = 32
impact.writeInputFile(beam,latticeB,'test.in.'+str(nTurn)+'.backward')